In [1]:
import numpy as np
import pandas as pd
from collections import Counter

In [2]:
class Node:
    def __init__(self,best_feature=None,best_threshold=None,left=None,right=None,value=None):
        self.best_feature = best_feature
        self.best_threshold = best_threshold
        self.left = left
        self.right = right
        self.value = value
    
    def _is_leaf_node(self):
        return self.value is not None
        

In [24]:
class DecisionTree:
    def __init__(self,max_depth=100,min_sample_split=2,n_features=None):
        self.max_depth = max_depth
        self.min_sample_split = min_sample_split
        self.n_features = n_features
        self.root = None
    
    def fit(self,X,y,depth=0):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1],self.n_features)
        
        self.root = self._grow_tree(X,y,depth)
    
    def _grow_tree(self,X,y,depth):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))
        
        #stopping criteria
        #depth > max_depth or labels ==1 or n_samples < min_sample_split
        if (depth >= self.max_depth or n_labels == 1 or n_samples < self.min_sample_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)
        
        #getting the features 
        feature_idxs = np.random.choice(n_features,self.n_features,replace=False)
        
        best_feature, best_threshold = self._best_split(X,y,feature_idxs)
        
        left_idx, right_idx = self._split(X[:,best_feature],best_threshold)
        
        left = self._grow_tree(X[left_idx,:],y[left_idx],depth+1)
        right = self._grow_tree(X[right_idx,:],y[right_idx],depth+1)
        
        return Node(best_feature=best_feature,best_threshold=best_threshold,left=left,right=right)
    
    
    def _best_split(self,X,y,feature_idxs):
        best_gain = -1
        split_feature, split_threshold = None,None
        
        for feature_id in feature_idxs:
            X_c = X[:,feature_id]
            thresholds = np.unique(X_c)
            
            for thr in thresholds:
                gain = self._information_gain(X_c,y,thr)
                
                if gain > best_gain:
                    best_gain = gain
                    split_feature = feature_id
                    split_threshold = thr
                    
        return split_feature,split_threshold
    
    def _information_gain(self,X,y,threshold):
        #entropy(parent) - weighted avg(entropy(children))
        
        parent_entropy = self._entropy(y)
        
        left_idxs, right_idxs = self._split(X,threshold)
        
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0
        
        n_l,n_r = len(left_idxs),len(right_idxs)
        e_l,e_r = self._entropy(y[left_idxs]),self._entropy(y[right_idxs])
        
        child_entropy = (n_l/len(y))*e_l + (n_r/len(y))*e_r
        
        
        return parent_entropy - child_entropy
    
    def _entropy(self,y):
        count = np.bincount(y)
        
        px = count/len(y)
        
        entropy = -np.sum([p * np.log(p) for p in px if p > 0])
        
        return entropy
    
    
    def _split(self,X,thr):
        left_idxs = np.argwhere(X<=thr).flatten()
        right_idxs = np.argwhere(X>thr).flatten()
        
        return left_idxs,right_idxs
        
            
    def _most_common_label(self,y):
        counter = Counter(y)
        most_common = counter.most_common(1)[0][0]
        return most_common
    
    def predict(self,X):
        preds = np.array([self._traverse_tree(x,self.root) for x in X])
        return preds
    
    def _traverse_tree(self,x,node):
        if node._is_leaf_node():
            return node.value
        
        if x[node.best_feature] <= node.best_threshold:
            return self._traverse_tree(x,node.left)
        
        return self._traverse_tree(x,node.right)
        

In [9]:
import sklearn
from sklearn import datasets

bc = datasets.load_breast_cancer()

X,y = bc.data,bc.target

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y)

In [25]:
clf = DecisionTree()
clf.fit(X_train,y_train)
preds = clf.predict(X_test)

In [26]:
acc = np.sum(y_test == preds)/len(y_test)
acc

0.951048951048951

In [4]:
x = [1,2,4,5,2,1,3]
np.bincount(x)

array([0, 2, 2, 1, 1, 1], dtype=int64)

In [7]:
np.array([[1,2,4,4]]).flatten()

array([1, 2, 4, 4])